## 1 Einleitung


### 1.1 Ziel

### 1.2 Vorgehen

## 2 Daten

### 2.1 Quellen

### 2.2 Import, Bereinigung und Transformation

#### Verkehrsdaten

Laden eines einzelnen Datensatz auf Tagesbasis als Basis für Data Cleaning

In [51]:
import pandas as pd

datei_pfad = "/Users/remofankhauser/Verkehrsdaten_2024/2024-01-01_istdaten.csv"
df_verkehr_raw = pd.read_csv(datei_pfad, sep=None, engine="python") #Laden der CSV-Datei

display(df_verkehr_raw.head())

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
0,01.01.2024,80:06____:17291:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17291,RB,NaN,RB,...,True,8500090,Basel Bad Bf,NaN,NaN,UNBEKANNT,01.01.2024 01:05,NaN,UNBEKANNT,False
1,01.01.2024,80:06____:70665:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,70665,RB,NaN,RB,...,False,8500090,Basel Bad Bf,NaN,NaN,NaN,01.01.2024 01:05,01.01.2024 01:06:00,PROGNOSE,False
2,01.01.2024,80:800631:17238:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17238,RB,NaN,RB,...,False,8500090,Basel Bad Bf,01.01.2024 07:05,01.01.2024 07:05:00,PROGNOSE,NaN,NaN,NaN,False
3,01.01.2024,80:800631:17239:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17239,RB,NaN,RB,...,False,8500090,Basel Bad Bf,NaN,NaN,NaN,01.01.2024 07:53,01.01.2024 07:53:00,PROGNOSE,False
4,01.01.2024,80:800631:17241:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17241,RB,NaN,RB,...,False,8500090,Basel Bad Bf,NaN,NaN,NaN,01.01.2024 08:53,01.01.2024 08:54:00,PROGNOSE,False


Übersicht der vorhandenen Spalten und Datentypen

In [52]:
display(df_verkehr_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1501831 entries, 0 to 1501830
Data columns (total 21 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   BETRIEBSTAG          1501831 non-null  object
 1   FAHRT_BEZEICHNER     1501831 non-null  object
 2   BETREIBER_ID         1501831 non-null  object
 3   BETREIBER_ABK        1501831 non-null  object
 4   BETREIBER_NAME       1501831 non-null  object
 5   PRODUKT_ID           1501831 non-null  object
 6   LINIEN_ID            1501831 non-null  object
 7   LINIEN_TEXT          1501831 non-null  object
 8   UMLAUF_ID            688303 non-null   object
 9   VERKEHRSMITTEL_TEXT  1501831 non-null  object
 10  ZUSATZFAHRT_TF       1501831 non-null  bool  
 11  FAELLT_AUS_TF        1501831 non-null  bool  
 12  BPUIC                1501831 non-null  int64 
 13  HALTESTELLEN_NAME    1389616 non-null  object
 14  ANKUNFTSZEIT         1413665 non-null  object
 15  AN_PROGNOSE    

None

Sparten welche offensichtlich nicht zur Analyse beitragen werden entfernt:
- **BETRIEBSTAG:** *Das Datum ist bereits in der Abfahrtszeit enthalten – daher überflüssig*
- **FAHRT_BEZEICHNER:** *Numerische Bezeichnung der Fahrt, die für die Analyse keinen Mehrwert bietet*
- **BETREIBER_ID:** *Numerische ID des Betreibers, redundant da BETREIBER_ABK ausreichend ist*
- **BETREIBER_NAME:** *Vollständiger Name des Betreibers, ebenfalls durch BETREIBER_ABK abgedeckt*
- **LINIEN_ID:** *Numerischer Identifikator der Linie, wobei LINIEN_TEXT klarere Informationen liefert*
- **UMLAUF_ID:** *Numerische Fahrtbezeichnung, die für die Analyse nicht relevant ist*
- **VERKEHRSMITTEL_TEXT:** *Abkürzung zu PRODUKT_ID – somit nicht erforderlich*
- **BPUIC:** *Nicht aussagekräftig für die Analyse*
- **ANKUNFTSZEIT:** *Wird nicht betrachtet, da ausschließlich Abfahrtszeiten relevant sind*
- **AN_PROGNOSE:** *Nicht genutzt, da nur Abfahrtszeiten im Fokus stehen*
- **AN_PROGNOSE_STATUS:** *Status der Ankunftsprognose wird nicht analysiert – daher entbehrlich*


In [53]:
df_verkehr = df_verkehr_raw.drop(columns=["BETRIEBSTAG", "FAHRT_BEZEICHNER", "BETREIBER_ID", "BETREIBER_NAME", "LINIEN_ID", "UMLAUF_ID", "VERKEHRSMITTEL_TEXT", "BPUIC", "ANKUNFTSZEIT", "AN_PROGNOSE", "AN_PROGNOSE_STATUS"])
display(df_verkehr.head())

,BETREIBER_ABK,PRODUKT_ID,LINIEN_TEXT,ZUSATZFAHRT_TF,FAELLT_AUS_TF,HALTESTELLEN_NAME,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
0,DB,Zug,RB,True,True,Basel Bad Bf,01.01.2024 01:05,NaN,UNBEKANNT,False
1,DB,Zug,RB,True,False,Basel Bad Bf,01.01.2024 01:05,01.01.2024 01:06:00,PROGNOSE,False
2,DB,Zug,RB,False,False,Basel Bad Bf,NaN,NaN,NaN,False
3,DB,Zug,RB,False,False,Basel Bad Bf,01.01.2024 07:53,01.01.2024 07:53:00,PROGNOSE,False
4,DB,Zug,RB,False,False,Basel Bad Bf,01.01.2024 08:53,01.01.2024 08:54:00,PROGNOSE,False


Filtern nach Daten der Zürcher Verkehrsbetriebe (VBZ) mit anschliessender Kontrolle und löschung der Spalte

In [ ]:
filter_betreiber = df_verkehr["BETREIBER_ABK"].astype(str).str.contains("VBZ", na=False)
df_verkehr_vbz = df_verkehr[filter_betreiber]

display(df_verkehr_vbz["BETREIBER_ABK"].value_counts()) #Kontrollen, ob nur VBZ drin sind

df_verkehr_vbz = df_verkehr_vbz.drop(columns=["BETREIBER_ABK"])


BETREIBER_ABK
VBZ           90362
VBZ    Fpl    69637
Name: count, dtype: int64

Analyse spezifischer Spalten auf ihre relevanz

In [ ]:
print(df_verkehr_vbz["AB_PROGNOSE_STATUS"].value_counts())
print((df_verkehr_vbz["ZUSATZFAHRT_TF"].value_counts() / len(df_verkehr_vbz)).round(4))
print((df_verkehr_vbz["FAELLT_AUS_TF"].value_counts() / len(df_verkehr_vbz)).round(4))
print(df_verkehr_vbz["DURCHFAHRT_TF"].value_counts() / len(df_verkehr_vbz))
print(df_verkehr_vbz["PRODUKT_ID"].value_counts())
#pberarbeiten

AB_PROGNOSE_STATUS
REAL          148627
PROGNOSE        1654
GESCHAETZT      1007
UNBEKANNT        313
Name: count, dtype: int64
ZUSATZFAHRT_TF
False    0.9876
True     0.0124
Name: count, dtype: float64
FAELLT_AUS_TF
False    0.9725
True     0.0275
Name: count, dtype: float64
DURCHFAHRT_TF
False    1.0
Name: count, dtype: float64
PRODUKT_ID
Bus     90362
Tram    69637
Name: count, dtype: int64


ZUSATZFAHRT und FAELLT_AUS haben einen TRUE Gehalt von 1.25 respektive 2.75%. Da dies eine relativ geringe grösse ist werden diese Eintröge als nicht relevant angesehen und gelöscht. Da es im VBZ-Netz scheinbar keine Durchfahrten gibt oder diese Werte nur sehr klein sind wird diese Spalte ebenfalls gelöscht. Der Prognose Status der Abfahrtszeit wird zudem nach Real und Geschätzt gefiltert da alle anderen aten nicht relevant für die Analyse sind (siehe Cookbook)

In [56]:
filter_status = df_verkehr_vbz["AB_PROGNOSE_STATUS"].astype(str).str.contains("REAL|GESCHAETZT", na=False)

filter_faellt_aus = df_verkehr_vbz["FAELLT_AUS_TF"] == False
filter_zusatzfahrt = df_verkehr_vbz["ZUSATZFAHRT_TF"] == False
filter_durchfahrt = df_verkehr_vbz["DURCHFAHRT_TF"] == False
df_verkehr_vbz = df_verkehr_vbz[filter_status & filter_faellt_aus & filter_zusatzfahrt & filter_durchfahrt]

df_verkehr_vbz = df_verkehr_vbz.drop(columns=["ZUSATZFAHRT_TF", "FAELLT_AUS_TF", "DURCHFAHRT_TF"])

display(df_verkehr_vbz.head())

,PRODUKT_ID,LINIEN_TEXT,HALTESTELLEN_NAME,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS
262646,Tram,2,"Zürich,Kalkbreite/Bhf.Wiedikon",01.01.2024 04:51,01.01.2024 04:52:03,REAL
262647,Tram,2,"Zürich, Bezirksgebäude",01.01.2024 04:52,01.01.2024 04:53:27,REAL
262648,Tram,2,"Zürich, Stauffacher",01.01.2024 04:54,01.01.2024 04:55:26,REAL
262649,Tram,2,"Zürich, Sihlstrasse",01.01.2024 04:55,01.01.2024 04:57:26,REAL
262650,Tram,2,"Zürich, Paradeplatz",01.01.2024 04:57,01.01.2024 04:59:19,REAL


Analyse der Verkehrsmittel und Linien

In [57]:
print(df_verkehr_vbz["PRODUKT_ID"].value_counts())
print(df_verkehr_vbz["LINIEN_TEXT"].unique())

PRODUKT_ID
Bus     83866
Tram    61525
Name: count, dtype: int64
['2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '17' '31'
 '32' '33' '35' '37' '38' '39' '40' '46' '61' '62' '64' '66' '67' '69'
 '70' '72' '73' '75' '77' '78' '80' 'EV' '89' '91' '99' '161' '163' '165'
 '184' '185' '301' '302' '304' '305' '306' '307' '309' '314' '317' '325'
 '701' '704' '743' '751' '910' '916' '919']


Nach Tram filtern um klaren Fokus für die Aufgabe zu schaffen

In [ ]:
filter_produkt = df_verkehr_vbz["PRODUKT_ID"]=="Tram"
df_verkehr_vbz = df_verkehr_vbz[filter_produkt]
print(df_verkehr_vbz["PRODUKT_ID"].unique))

df_verkehr_vbz = df_verkehr_vbz.drop(columns=["PRODUKT_ID"])

display(df_verkehr_vbz.head())

,LINIEN_TEXT,HALTESTELLEN_NAME,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS
262646,2,"Zürich,Kalkbreite/Bhf.Wiedikon",01.01.2024 04:51,01.01.2024 04:52:03,REAL
262647,2,"Zürich, Bezirksgebäude",01.01.2024 04:52,01.01.2024 04:53:27,REAL
262648,2,"Zürich, Stauffacher",01.01.2024 04:54,01.01.2024 04:55:26,REAL
262649,2,"Zürich, Sihlstrasse",01.01.2024 04:55,01.01.2024 04:57:26,REAL
262650,2,"Zürich, Paradeplatz",01.01.2024 04:57,01.01.2024 04:59:19,REAL


Datentypen anpassen

In [59]:
df_verkehr_vbz["ABFAHRTSZEIT"] = pd.to_datetime(df_verkehr_vbz["ABFAHRTSZEIT"]+ ":00", dayfirst=True)
df_verkehr_vbz["AB_PROGNOSE"] = pd.to_datetime(df_verkehr_vbz["AB_PROGNOSE"], dayfirst=True)

display(df_verkehr_vbz.info())

<class 'pandas.core.frame.DataFrame'>
Index: 61525 entries, 262646 to 332281
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   LINIEN_TEXT         61525 non-null  object        
 1   HALTESTELLEN_NAME   61525 non-null  object        
 2   ABFAHRTSZEIT        61525 non-null  datetime64[ns]
 3   AB_PROGNOSE         61525 non-null  datetime64[ns]
 4   AB_PROGNOSE_STATUS  61525 non-null  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 2.8+ MB


None

Aus den erkenntnissen der Tagesdaten wird nachfolgend ein Code erstellt der die Datensätze von einem Jahr zusmmenführt. Dabei werden die gleichen Methoden wie oben angewendet

In [ ]:
import glob
import os

# ----------Individuell Anpassen-----------
ordner_pfad = "/Users/remofankhauser/Verkehrsdaten_2024" # Ordnerpfad mit den CSV-Dateien
# -----------------------------------------

csv_dateien = glob.glob(os.path.join(ordner_pfad, "*.csv")) # Alle CSV-Dateien im Ordner finden

df_list = [] # Liste zum Speichern der DataFrames


for datei in csv_dateien:
    df = pd.read_csv(datei, sep=None, engine="python")  # Alle CSV-Dateien einlesen

    # Nicht benötigte Spalten entfernen
    df = df.drop(columns=["BETRIEBSTAG", "FAHRT_BEZEICHNER", "BETREIBER_ID", 
                          "BETREIBER_NAME", "LINIEN_ID", "UMLAUF_ID", 
                          "VERKEHRSMITTEL_TEXT", "BPUIC", "ANKUNFTSZEIT",
                          "AN_PROGNOSE", "AN_PROGNOSE_STATUS"])
    

    # Filterbedingungen definieren
    filter_conditions = (
        df["BETREIBER_ABK"].astype(str).str.contains("VBZ", na=False) &
        df["AB_PROGNOSE_STATUS"].astype(str).str.contains("REAL|GESCHAETZT", na=False) &
        (df["ZUSATZFAHRT_TF"] == False) & #Evtl erst später filtern
        (df["FAELLT_AUS_TF"] == False) & #Evtl erst später filtern
        (df["DURCHFAHRT_TF"] == False) & #Evtl erst später filtern
        df["PRODUKT_ID"].astype(str).str.contains("Tram", na=False)
    )

    df = df[filter_conditions]

    # # Kontrolle der Spalteninhalte
    # print(df["BETREIBER_ABK"].unique())
    # print(df["PRODUKT_ID"].unique())
    # print(df["ZUSATZFAHRT_TF"].unique())
    # print(df["FAELLT_AUS_TF"].unique())
    # print(df["DURCHFAHRT_TF"].unique())


    # Weitere Spalten entfernen
    df = df.drop(columns=["BETREIBER_ABK", "PRODUKT_ID", "ZUSATZFAHRT_TF","FAELLT_AUS_TF", "DURCHFAHRT_TF"])


    # Datums- und Zeitformat korrekt setzen
    df["ABFAHRTSZEIT"] = pd.to_datetime(df["ABFAHRTSZEIT"]+ ":00", dayfirst=True)# format="%d.%m.%Y %H:%M:%S")
    df["AB_PROGNOSE"] = pd.to_datetime(df["AB_PROGNOSE"], dayfirst=True)# format="%d.%m.%Y %H:%M:%S")


    # df_list.append(df)
    # display(df.head())
    # break

    if df_list: # prüfen ob df_list leer ist
        df_gesamt = pd.concat(df_list, ignore_index=True) # Alle DataFrames zusammenfügen
    
        df_gesamt = df_gesamt.sort_values(by="ABFAHRTSZEIT").reset_index(drop=True)  # Sortierung nach Abfahrtszeit und Index zurücksetzen

        # speicherpfad = os.path.join(ordner_pfad, "Verkehrs_Daten_2024.csv") # Speicherpfad definieren
        # df_gesamt.to_csv(speicherpfad, index=False) # DataFrame speichern

,LINIEN_TEXT,HALTESTELLEN_NAME,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS
269250,2,"Zürich, Kappeli",2024-07-28 17:00:00,2024-07-28 17:01:48,REAL
269251,2,"Zürich, Freihofstrasse",2024-07-28 17:01:00,2024-07-28 17:02:53,REAL
269252,2,"Zürich, Letzigrund",2024-07-28 17:03:00,2024-07-28 17:04:11,REAL
269253,2,"Zürich, Albisriederplatz",2024-07-28 17:04:00,2024-07-28 17:05:34,REAL
269254,2,"Zürich, Zypressenstrasse",2024-07-28 17:05:00,2024-07-28 17:07:04,REAL


In [ ]:
df_2024 = pd.read_csv("/Users/remofankhauser/Verkehrsdaten_2024/Verkehrs_Daten_2024_red.csv", sep=None, engine="python")
print(df_2024["LINIEN_TEXT"].value_counts())
print((df_verkehr_vbz["ZUSATZFAHRT_TF"].value_counts() / len(df_verkehr_vbz)).round(4))
print((df_verkehr_vbz["FAELLT_AUS_TF"].value_counts() / len(df_verkehr_vbz)).round(4))
print(df_verkehr_vbz["DURCHFAHRT_TF"].value_counts() / len(df_verkehr_vbz))

In [ ]:
filter_linie_10 = df_2024["LINIEN_TEXT"] != "10"
filter_linie_E = df_2024["LINIEN_TEXT"] != "E"

df_2024_red = df_2024[filter_linie_10 & filter_linie_E]

In [ ]:
df_2024.drop(columns=["ZUSATZFAHRT_TF", "FAELLT_AUS_TF", "DURCHFAHRT_TF"], inplace=True)

display(df_2024.head())

### Wetterdaten

## 3 Datenanalyse

## 4 Präsentation der Ergebnisse